# Análisis de posibilidad de atención a clientes

Este cuaderno es un análisis de la posibilidad de atender a los clientes de acuerdo a una segmentación dada por un conjunto de condiciones: cumplimiento de 15% de saldo, nivel de RCI, tipo de oferta, etc.

## Condiciones actuales de precio

In [25]:
# importar librerias
import pandas as pd
import utils as u

In [19]:
# leer datos
df = pd.DataFrame.from_dict({"CODIGO CLIENTE": [1, 2],
                                "NO CREDITO": [1, 2],
                                "RCI": [25, 48], "DEUDA EXTERNA": [100000, 120000], 
                                "SALDO INTERNO": [500000, 60000],
                                "TASA": [15.6, 19.3], "PLAZO": [120, 108], 
                                "MONTO": [25000, 12000], "SALDO": [0.1, 0.6],
                                "SALARIO": [10000, 8000]})
# quedarse con las columnas útiles: rci, deuda externa, saldo interno, tasa, plazo, monto 

# tratamiento general de datos
df["CUMPLE 15%"] = df["SALDO"].apply(u.cumple_15)

# calcular saldo posible a consolidacion
datos_agrupado = df.groupby(by="CODIGO CLIENTE")["SALDO"].sum().reset_index()
datos_agrupado.columns = ["CODIGO CLIENTE", "SALDO INTERNO TOTAL CLIENTE"]
df = pd.merge(df, datos_agrupado, how="left", on="CODIGO CLIENTE")

datos_agrupado = df.groupby(by="CODIGO CLIENTE")["DEUDA EXTERNA"].sum().reset_index()
datos_agrupado.columns = ["CODIGO CLIENTE", "SALDO EXTERNO TOTAL CLIENTE"]
df = pd.merge(df, datos_agrupado, how="left", on="CODIGO CLIENTE")

# sacar saldo total
df["SALDO TOTAL CLIENTE"] = df["SALDO INTERNO TOTAL CLIENTE"] + df["SALDO EXTERNO TOTAL CLIENTE"]


In [16]:
# segmentar clientes con el cumplimiento: segmentar por crédito y luego ver opciones de recrédito agrup
datos = df.copy()

# sacar el monto máximo para llegar a 50% de RCI con la tasa actual más 1
datos["MONTO RCI50 CON TASA ACTUAL +1"] = datos.apply(lambda x: u.calcular_monto_a_rci(x["TASA"]+1, 50, x["PLAZO"], x["RCI"], x["SALARIO"]), axis=1)
# sacar el motno máximo para llegar a 60% de RCI con tasa actual más 1
datos["MONTO RCI60 CON TASA ACTUAL +1"] = datos.apply(lambda x: u.calcular_monto_a_rci(x["TASA"]+1, 60, x["PLAZO"], x["RCI"], x["SALARIO"]), axis=1)

# colocar bandera de si se puede consolidar con esa tasa con 50 y 60 de RCI
datos["FLAG CONSOLIDA CON RCI50"] = datos.apply(lambda x: u.comparar_montos(x["SALDO TOTAL CLIENTE"], x["MONTO RCI50 CON TASA ACTUAL +1"]), axis=1)
datos["FLAG CONSOLIDA CON RCI60"] = datos.apply(lambda x: u.comparar_montos(x["SALDO TOTAL CLIENTE"], x["MONTO RCI60 CON TASA ACTUAL +1"]), axis=1)


# ingresos a 6 meses con montos máximos aunque no sean consolidaciones
datos["INGRESO A 6 MESES MONTO RCI 50"] = datos.apply(lambda x: u.ingresos_financieros(x["MONTO RCI50 CON TASA ACTUAL +1"], x["TASA"] + 1, x["PLAZO"], 6), axis=1)
datos["INGRESO A 6 MESES MONTO RCI 60"] = datos.apply(lambda x: u.ingresos_financieros(x["MONTO RCI60 CON TASA ACTUAL +1"], x["TASA"] + 1, x["PLAZO"], 6), axis=1)

# sacar ingresos a 6 meses solo para los que pueden consolidar a al menos uno de los anteriores
datos["INGRESO A 6 MESES EN CONSOLIDACION"] = datos.apply(lambda x: u.ingresos_financieros(x["SALDO TOTAL CLIENTE"], x["TASA"] + 1, x["PLAZO"], 6), axis=1)


### Reservas y reestructuración: cálculo con máscaras

In [ ]:
mascara_reserva = datos["CUMPLE 15%"] == 0
datos.loc[mascara_reserva, "RESERVAS REESTRUCTURA MONTO RCI50 TASA ACTUAL+1"] = datos.loc[mascara_reserva, "MONTO RCI50 CON TASA ACTUAL +1"].apply(u.reservas_reestructuracion)
datos.loc[mascara_reserva, "RESERVAS REESTRUCTURA MONTO RCI60 TASA ACTUAL+1"] = datos.loc[mascara_reserva, "MONTO RCI60 CON TASA ACTUAL +1"].apply(u.reservas_reestructuracion)
datos.loc[mascara_reserva, "RESERVAS CONSOLIDACION TASA ACTUAL +1"] = datos.loc[mascara_reserva, "SALDO TOTAL CLIENTE"].apply(u.reservas_reestructuracion)


# compara que clientes cubren reservas en los 6 meses
datos.loc[mascara_reserva, "CUMPLE RESERVAS CON INGRESOS RCI50 TASA ACTUAL +1"] = datos.loc[mascara_reserva].apply(lambda x: u.comparar_montos(x["RESERVAS REESTRUCTURA MONTO RCI50 TASA ACTUAL+1"], x["INGRESO A 6 MESES MONTO RCI 50"]), axis=1)
datos.loc[mascara_reserva, "CUMPLE RESERVAS CON INGRESOS RCI60 TASA ACTUAL +1"] = datos.loc[mascara_reserva].apply(lambda x: u.comparar_montos(x["RESERVAS REESTRUCTURA MONTO RCI60 TASA ACTUAL+1"], x["INGRESO A 6 MESES MONTO RCI 60"]), axis=1)
datos.loc[mascara_reserva, "CUMPLE RESERVAS CON INGRESOS CONSOLIDACION TASA ACTUAL +1"] = datos.loc[mascara_reserva].apply(lambda x: u.comparar_montos(x["SALDO TOTAL CLIENTE"], x["INGRESO A 6 MESES EN CONSOLIDACION"]), axis=1)

## Condiciones de precio máximo

In [ ]:
# calcular tasa minima para dejar a cero
datos["TASA MINIMA"] = datos.apply(lambda x: u.tasa_minima(reestructura=True))

# sacar tasa maxima para consolidar
datos["TASA MAXIMA CONSOLIDACION"] = datos.apply(lambda x: u.tasa_maxima(x["RCI"], 60), axis=1)
datos["TASA MAXIMA LIQUIDEZ"] = datos.apply(lambda x: u.tasa_maxima(x["RCI"], 50), axis=1)

# comparar si el intervalo tiene sentido
datos["POSIBLE LIQUIDEZ PRECIO PERSONALIZADO"] = datos.apply(lambda x: u.comparar_montos(x["TASA MINIMA"], x["TASA MAXIMA LIQUIDEZ"]), axis=1)
datos["POSIBLE CONSOLIDACION PRECIO PERSONALIZADO"] = datos.apply(lambda x: u.comparar_montos(x["TASA MINIMA"], x["TASA MAXIMA CONSOLIDACION"]), axis=1)